## Budget agent with langachin

In [9]:
# LangChain Budget Recommendation Agent
# This LangChain tool takes user transaction summaries and returns budget recommendations

from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
# from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama

from langchain.tools import BaseTool
from typing import Callable
import json
import ollama
from ollama import Client


# -- Step 1: Define the function to generate the budget plan --
def generate_budget_plan(transactions_summary: dict) -> str:
    prompt = f"""
    You are a financial AI assistant.
    Analyze the transaction summary below and generate a personalized monthly budget plan.

    Include:
    1. Budget Overview with amount allocation (Income, Essentials, Flex, Savings)
    2. Key Areas of Overspending
    3. Personalized Savings Tips

    === TRANSACTION SUMMARY ===
    {json.dumps(transactions_summary, indent=2)}

    Respond in this format:
    ---
    Monthly Budget Overview:
    - Income:
    - Essentials:
    - Flexible Spending:
    - Savings:

    Key Areas of Overspending:
    - ...

    Savings Tips:
    - ...
    ---
    """
    # llm = ChatOpenAI(temperature=0, model="gpt-4")
    llm = Ollama(model="deepseek-r1")
    response = llm.invoke(prompt)
    return response.strip()


# -- Step 2: Wrap the function into a LangChain Tool --
class BudgetRecommendationTool(BaseTool):
    name: str = "budget_recommendation_tool"
    description: str = "Use this tool to get a personalized budget recommendation from transaction summaries"
    func: Callable[[dict], str]

    def _run(self, transactions_summary: dict) -> str:
        return self.func(transactions_summary)

    def _arun(self, transactions_summary: dict):
        raise NotImplementedError("This tool does not support async")


# -- Step 3: Define the Agent with the Tool --
tool = BudgetRecommendationTool(func=generate_budget_plan)
tools = [tool]

agent = initialize_agent(
    tools=tools,
    # llm=ChatOpenAI(temperature=0, model="gpt-4"),
    llm = Ollama(model="deepseek-r1"),

    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# -- Step 4: Use the agent with sample data --
sample_data = {
    'starting_balance': 5000,
    'ending_balance': 6200,
    'total_deposited': 3000,
    'total_withdrawn': 1800,
    'average_monthly_balance': 5600,
    'monthly_summary': [],
    'category_breakdown': [
        {'category': 'Groceries', 'amount': 600},
        {'category': 'Entertainment', 'amount': 400},
        {'category': 'Rent', 'amount': 1000}
    ]
}

response = agent.run(f"Analyze this transaction summary and generate a monthly budget: {json.dumps(sample_data)}")
print(response)




> Entering new AgentExecutor chain...
<think>
Okay, let me tackle this query. The user wants a monthly budget based on the given transaction summary.

First, I see that they provided specific data like starting balance, ending balance, total deposited, and withdrawn amounts. But the main thing here is the category breakdown with three categories: Groceries, Entertainment, and Rent. Hmm, those are just some of the common expenses people have. The user might not have listed all their spending categories yet.

I should use the budget_recommendation_tool because it's designed to handle transaction summaries and generate personalized budgets. But before that, I need to figure out what other information is needed from the tool. Since there's no monthly_summary in the input (it's an empty list), maybe the tool expects more details like income sources or other expenses not listed here.

The user might be expecting a comprehensive budget that includes all aspects of their finances, but they h

## AI Budgeting Agent - Prompt Tuning Notebook
Use this notebook to experiment with prompts that generate personalized budget recommendations
based on a user's transaction summary. You can iterate on prompt wording and review model responses.

In [ ]:
import json
import ollama
from ollama import Client

# Sample transaction summary data (replace with actual analysis output)
transactions_summary = {
    'starting_balance': 5000,
    'ending_balance': 6200,
    'total_deposited': 3000,
    'total_withdrawn': 1800,
    'average_monthly_balance': 5600,
    'monthly_summary': [],
    'category_breakdown': [
        {'category': 'Groceries', 'amount': 600},
        {'category': 'Entertainment', 'amount': 400},
        {'category': 'Rent', 'amount': 1000}
    ]
}

prompt = f"""
You are a financial AI assistant.
Analyze the transaction summary below and generate a personalized monthly budget plan.

Include:
1. Budget Overview with amount allocation (Income, Essentials, Flex, Savings)
2. Key Areas of Overspending
3. Personalized Savings Tips

=== TRANSACTION SUMMARY ===
{json.dumps(transactions_summary, indent=2)}

Respond in this format:
---
Monthly Budget Overview:
- Income:
- Essentials:
- Flexible Spending:
- Savings:

Key Areas of Overspending:
- ...

Savings Tips:
- ...
---
"""

In [9]:
# Generate response using Ollama
client = Client()
response = client.generate(
    model='deepseek-r1',
    prompt=prompt
)
print(response.response.strip())

<think>
Hmm, the user is asking me to act as a financial AI assistant and analyze their transaction summary to create a personalized monthly budget plan. They want three specific sections: Budget Overview with income allocation, key overspending areas, and savings tips.

Looking at the data provided in the query, I can see they started with $5000 and ended with $6200 for the month - that's positive growth! Their total deposited was $3000 (likely their salary or main income) and total withdrawn was $1800. Wait, if they deposited $3000 and withdrew only $1800, then where did the extra money come from? The difference between ending balance ($6200) minus starting balance ($5000) is indeed their deposit of $3000 plus some unaccounted income or gains. But I notice they didn't provide a breakdown for all categories - just groceries (600), entertainment (400), and rent (1000). There must be other expenses that aren't listed.

For the budget overview, I'll need to calculate what's left in their